In [ ]:
import tkinter as tk
from tkinter import ttk, Canvas
import time
import funciones as fn
import threading
from os.path import isfile
import csv
import datetime

In [ ]:
#Funciones
def conectar():
    """Gestiona la conexión con el servidor y los estados"""
    global estados, cn
    txt=app.btn_conectar['text']
    if txt=='CONECTAR SRV':
        try:
            srv.conectar(host="192.168.99.10", port=9999)
            hs=""
            hs=srv.enviar("hs")
            if hs=="hs":
                #app.lbl_sts.config(text="Conectado con servidor")
                srv.status="Conectado"
                #app.lbl_stssrv.config(text="Conectado")
                app.btn_conectar['text']="DESCONECTAR SRV"
            else:
                srv.status="Error"
                #app.lbl_stssrv.config(text="Error")
        except Exception as e:
            srv.status=f"Desconectado con error {e}"
            #app.lbl_stssrv.config(text="Desconectado con error")
    else:
        srv.cerrar()
        srv.status="Desconectado"
        app.btn_conectar['text']="CONECTAR SRV"
        cn=0
    estados['srvcon']=srv.status
    app.lbl_stssrv.config(text=srv.status)
    
def inicializar():
    global LCR
    LCR.inicializar('COM3', 19200, 8, 'N', 1)
    LCR.inicializado=True
    


def configurar():
    global estados, tfrec, ifrec, repeticion, configuracion
    dic={True:"OK", False: "NO OK"}
    ac=fn.configurar()
    configuracion=ac.cargarconfig("configuracion/config.txt")
    estados['config']=f"Frec: {dic[configuracion['okfrec']]}, rep: {dic[configuracion['okrep']]}"
    app.lbl_stsconf.config(text=estados['config'])
    app.lbl_gral.config(text=str(configuracion), **neutro)
    tfrec=len(configuracion['frecuencias'])
    ifrec=0
    repeticion=0
    

def inimed():
    "Inicia la medicion"
    global flag_medIni
    flag_medIni=True
    app.lbl_stsmed.config(text="Medición iniciada")

def detener():
    "Detiene la medicion"
    global flag_medIni
    flag_medIni=False
    app.lbl_stsmed.config(text="Medición detenida")    

#funcion para guardar una fila en csv.
#si el archivo no existe, te lo crea
def escribir_csv(archivo:str, datos:list, encabezado:list):
    """
    Abre un archivo CSV en modo 'append', por lo que agrega una nueva fila.
    archivo     -> string. Formato 'carpeta\\archivo.csv' ('Mediciones\probando.csv')
    fila        -> lista con los valores a guardar
    encabezado  -> Lista con los textos de los encabezados
    """

    aux=isfile(archivo)
    
    with open(archivo, mode='a', newline='',) as archivo_csv:
        writer = csv.writer(archivo_csv, delimiter=";")
        if not aux:
            writer.writerow(encabezado)
        writer.writerow(datos)
        
    archivo_csv.close()

In [ ]:
class App(threading.Thread):

    def __init__(self):
        threading.Thread.__init__(self)
        #self.daemon=True
        self.start()

    def callback(self):
        self.root.quit()

    def run(self):
        self.root = tk.Tk()
        self.root.protocol("WM_DELETE_WINDOW", self.callback)
        self.root.title("ADQUISIDOR CAPEM")
        self.root.config(width=497, height=390)

        self.lbl_comandos = ttk.Label(text="COMANDOS:",font=("Arial", 13))
        self.lbl_comandos.place(x=20, y=20)

        #BOTONES
        self.btn_configurar= ttk.Button(text="CONFIGURAR", command=configurar)
        self.btn_configurar.place(x=20, y=50,width=100, height=30)

        self.btn_conectar= ttk.Button(text="CONECTAR SRV", command=conectar)
        self.btn_conectar.place(x=20, y=80,width=100, height=30)

        self.btn_inicializar= ttk.Button(text="INICIALIZAR LCR", command=inicializar)
        self.btn_inicializar.place(x=20, y=110,width=100, height=30)

        self.btn_medir= ttk.Button(text="MEDIR", command=inimed)
        self.btn_medir.place(x=20, y=140,width=100, height=30)

        self.btn_detener= ttk.Button(text="DETENER", command=detener)
        self.btn_detener.place(x=20, y=170,width=100, height=30)


        #ETIQUETAS FIJAS DE ESTADO
        self.lbl_comandos = ttk.Label(text="ESTADOS:",font=("Arial",13))
        self.lbl_comandos.place(x=20, y=240)

        self.lbl_sts_srv=ttk.Label(text="Servidor:")
        self.lbl_sts_srv.place(x=20, y=265)

        self.lbl_sts_serie=ttk.Label(text="Instrumento:")
        self.lbl_sts_serie.place(x=20, y=285)

        self.lbl_sts_conf=ttk.Label(text="Configuracion:")
        self.lbl_sts_conf.place(x=20, y=305)

        self.lbl_sts_med=ttk.Label(text="Medición:")
        self.lbl_sts_med.place(x=20, y=325)

        #ETIQUETAS variables DE ESTADO
        self.lbl_stssrv=ttk.Label(text="Desconectado")
        self.lbl_stssrv.place(x=130, y=265)

        self.lbl_stsserie=ttk.Label(text="Desconectado")
        self.lbl_stsserie.place(x=130, y=285)

        self.lbl_stsconf=ttk.Label(text="Sin configurar")
        self.lbl_stsconf.place(x=130, y=305)

        self.lbl_stsmed=ttk.Label(text="Detenida")
        self.lbl_stsmed.place(x=130, y=325)

        
        
        
        #Indicadores para operador
        self.canvas=Canvas(width=190, height=150)
        self.canvas.configure(background="gray")
        self.canvas.pack()
        self.canvas.place(x=155, y=50)

        self.lbl_gral=ttk.Label(text="...",font=("Arial",11,'bold'), padding=2, relief="ridge",width=61)
        self.lbl_gral.place(x=0, y=363)      

        self.root.mainloop()
    
    

In [ ]:
flag_medIni=False       #Flag para iniciar medicion desde HMI (A)
flag_posOK=False        #Flag para indicar que la posicion está OK (B). Se actualiza mediante OPC
flag_comOK= False       #Flag que indic que la comunicación es actual y está OK (C)
flag_cicloFin=False     #Flag para indicar cuándo finalizó las n repeticiones con todas las frecuencias

configuracion={}        #en este diccinario se cargan las configuraciones. Las claves son:
                        #'frecuencias' (lista), 'repeticiones' (int), 'okfrec' (bool) y 'okr' (bool)

srv=fn.conexionsrv()    #Genero el objeto servidor, que tiene todas las funciones para conectarse
cn=0                    #cn es el numero de control que se usa para chequear que la info leida  de srv es la correcta 
LCR=fn.LCR()            #creo el objeto lcr

#Variables que se van actualizando con los barridos de frecuencia
tfrec=0                 #cantidad de frecuencias totales en la config
ifrec=0                 #Indice de frecuencia actual
repeticion=0            #repeticion actual
filtroPos01=0           #Fitro para determinar el cambio de posicion OK a no OK
filtroPos10=0           #Fitro para determinar el cambio de posicion no OK a OK

estados=dict(srvcon="Desconectado",sercon="Desconectado",   #diccionario  para indicar los estados de las conexiones y funciones
             config="Sin configurar", medicion="Detenido")

encabezado=['Repetición', 'Frecuencia','En posicion','Posicion', 'Error SC', 'Presion', 'Nivel agua', 
            'T recipiente', 'T subenfriada', 'T Estimada', 'Inductancia', 'Resistencia', 'Dia', 'Hora']

datos_guardar=[]

#Diccionarios creados para dar estilos a los labels
verde={'background':'#4BF24B', 'foreground':'black'}
rojo={'background':'red', 'foreground':'white'}
amarillo={'background':'#ffe000', 'foreground':'black'}
neutro={'background':'#f0f0f0', 'foreground':'black'}

In [ ]:
app = App()

In [ ]:
while app.is_alive():       #Se finaliza cuando la interfaz se cierra
    if flag_medIni:         #Iniciar medición desde HMI (A)
        #if not LCR.inicializado or srv.status
        app.canvas.config(background="#ffe000")
        app.lbl_gral.config(text='Midiendo, no mover carajo',**amarillo)
        
        #Pido datos a OPC:
        cn+=1               #mando un numero de confirmación para que lo devuelva y saber si el valor devuelto es actual
        if cn>999: cn=0
        dts=srv.enviar(str(cn))                     #Me devuelve los datos solicitados
        datosOPC=list(dts[1:-1].split(sep=","))     #Hago un arreglo para quitarle los corchetes de la lista
        for n,x in enumerate(datosOPC):             #y los convierto en numeros 
            if n>1:                           
                datosOPC[n]=float(x)
            else:
                datosOPC[n]=int(x)             #el primero queda entero porque lo comparo con cn

        
        #Analizo el dato de posición traido de OPC. Si está en posición, se activa un flag (B)
        if datosOPC[1]==1:         #El bit de posición tiene que estar en el lugar 1 de la lista enviada
            filtroPos01+=1
            if filtroPos01>=10:     #Espera un tiempo... considerando las demoras, filtroPos01>=10 es un poco mas de 5 segundos
                flag_posOK=True
        else:
            flag_posOK=False
            filtroPos01=0
        
        #Comparo el numero de confirmación para chequear comunicacion (C)
        if datosOPC[0]==cn and datosOPC[0]>0:        #Activa el flag comOK si esta todo OK
            flag_comOK=True
            app.lbl_stssrv.config(text="Conectado")
        else:
            flag_comOK=False    #Si no lo desactiva y deja un mensaje en HMI
            app.lbl_stssrv.config(text="Error de datos")
            app.lbl_gral.config(text="Error de datos desde OPC",**rojo)
        

        #Chequeo cuándo se cumplen las n repeticiones de los barridos de frecuencia (D)
        if repeticion>=configuracion['repeticiones']:
            flag_cicloFin=True

        if flag_cicloFin:           #Si se finaliza el ciclo...
            repeticion=0
            ifrec=0        
            if not flag_posOK:      #...Espera a que salga de la posición para reiniciarlo
                filtroPos10+=1
                if filtroPos10>=10:    #Espera un tiempo... considerando las demoras, filtroPos10>=10 es un poco mas de 5 segundos
                    flag_cicloFin=False
                    filtroPos10=0
            else: filtroPos10=0

        
    
        # Chequeo que se den las condiciones para medir (A=1 -siempre está porque es el if principal- , B=1 ,C=1 y D=0)
        if flag_posOK and flag_comOK and not flag_cicloFin:
                        
            #Si se cumplen las condiciones, empiezo a medir

            #Colores y texto de etiqueta
            app.canvas.config(background="red")
            app.lbl_gral.config(text='Midiendo, no mover', **rojo)
            
            #Primero traigo la frecuencia de las configuraciones
            frecuencia=configuracion['frecuencias'][ifrec]     #no hago un 'for x in frecuencias' porque no pararía hasta terminar todos los valores
            LCR.cambiarfreq(frecuencia)
            time.sleep(1)
            datosLCR=LCR.medir()
            
            #armo la cadena de datos para guardar, con los datos de OPC y de equipo a medir
            fecha=[str(datetime.datetime.now().date()), str(datetime.datetime.now().time())]
            
            datos_guardar=[repeticion, frecuencia]       #datos básicos
            datos_guardar=datos_guardar+datosOPC[1:]     #Datos de OPC a partir del 1 (el cero es el check de comunicacion)
            datos_guardar=datos_guardar+datosLCR         #Datos de LCR
            datos_guardar=datos_guardar+fecha            #Fecha y hora

            #Guardo los datos en csv
            escribir_csv('Mediciones\datos.csv',datos_guardar,encabezado)
            ifrec+=1
            if ifrec>=tfrec:        #límite para las frecuencias
                ifrec=0
                repeticion+=1
                
        #Colores y textos para la etiqueta general
        if flag_cicloFin:
            app.canvas.config(background="#4BF24B")
            app.lbl_gral.config(text='Fin de medición, se puede mover', **verde)
        
        if flag_posOK and not flag_cicloFin:
            app.canvas.config(background="#ffe000")
            app.lbl_gral.config(text='En movimiento', **amarillo)


            
        if datosOPC[1]==1.0:
            app.canvas.config(background="red")
        else:
            app.canvas.config(background="green")
        
    else:
        app.canvas.config(background="gray")

#Cuando cierra la aplicacion, manda el comando de cerrar el servidor para que no de problemas
srv.cerrar()
        